In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from finta import TA
from finta.utils import resample_calendar
import json
import hvplot.pandas
import holoviews as hv
import panel as pn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split



pn.extension()
hv.extension('bokeh')



In [ ]:
# Instantiating the crypto client
client = CryptoHistoricalDataClient()
# Setting a start date and end date
#start_date = pd.Timestamp('2020-01-01', tz='America/New_York').isoformat()
#end_date = pd.Timestamp('2023-01-01', tz='AMerica/New_York').isoformat()
# Setting the tickers
#ticker = ['BTC/USD']

# Setting timeframe to '4Hour' for Alpaca API
#timeframe = '4Hour'

# Getting current ohlcv for BTC/USD
request_params = CryptoBarsRequest(
    symbol_or_symbols=['BTC/USD'],
    timeframe=TimeFrame.Hour,
    start='2020-01-01 00:00:00',
    end='2023-01-01 00:00:00'
)

# Retreiving the $ hourly bars for BTC/USD
btc_bars_48months = client.get_crypto_bars(request_params)

# Converting to a DataFrame
btc_bars_48months.df

In [ ]:
# Converting json to a dataframe
btc_bars_48months_df = btc_bars_48months.df
# Dropping columns
ohlcv_df  = btc_bars_48months_df.drop(columns=['trade_count','vwap'])
# Renaming the 'timestamp' column to the 'date' column and setting it as an index
ohlcv_df = ohlcv_df.reset_index()
ohlcv_df['date'] = ohlcv_df['timestamp']
ohlcv_df = ohlcv_df.drop(columns=['timestamp','symbol'])
ohlcv_df.set_index('date', drop=True, inplace=True)
# Making the df into a csv file
ohlcv_df.to_csv('ohlcv_BTC.csv')





-----
### Now I'm ready to manipulate the dataframe to build new columns like an RSI, RSI SMA and volume SMA.
-----

In [ ]:
# Reading in the csv to a df
ohlcv_df = pd.read_csv(
    Path('./Resources/ohlcv_BTC.csv'),
    index_col='date',
    infer_datetime_format=True,
    parse_dates=True
)

# Resampling the df to be 4hr candles
four_hr_ohlcv = resample_calendar(ohlcv_df, '4h')
four_hr_ohlcv.dropna()

In [ ]:
# Starting out my signals dataframe
signals_df = four_hr_ohlcv.loc[:,['close']]
# Generating returns from the BTC close prices using pct_change
signals_df['Actual_Returns'] = signals_df['close'].pct_change().dropna()
signals_df['Signal'] = 0.0
signals_df.loc[(signals_df['Actual_Returns'] >= 0), 'Signal'] = 1
signals_df.loc[(signals_df['Actual_Returns'] < 0), 'Signal'] = -1

(1 + signals_df[['Actual_Returns']]).cumprod().hvplot()





In [ ]:
# Adding more indicators to the sisgnals_df
signals_df['RSI14'] = TA.RSI(four_hr_ohlcv,14)
signals_df.dropna()
# Checking the RSI plot
RSI63 = signals_df['RSI14'].hvplot()

### Making the volume SMA

In [ ]:
# making the sma  rolling window for volume
volume_sma = 3
signals_df['volume'] = four_hr_ohlcv['volume']

# making the sma based on the rolling window for volume
signals_df['Volume_SMA_7'] = signals_df['volume'].rolling(window=volume_sma).mean()
signals_df.dropna()

# Checking the Volume SMA plot
Volume_SMA_7 = signals_df['Volume_SMA_7'].hvplot(kind='line')
Volume = signals_df['volume'].hvplot.bar()
display(Volume) 
display(Volume_SMA_7)




### Making the RSI SMA 


In [ ]:
# Setting the window for the RSI SMA
RSI_SMA = 7

# Making the RSI SMA based on the rolling window
signals_df['RSI_SMA_7'] = signals_df['RSI14'].rolling(window=RSI_SMA).mean()
signals_df.dropna()

RSI_SMA_7 = signals_df['RSI_SMA_7'].hvplot()


In [ ]:
# Plotting bot indicators for review
RSI63 * RSI_SMA_7

In [ ]:
# Sampling the df
signals_df.sample

In [ ]:
# Creating a signal for the actual strategy
signals_df['A1_Strategy_Signal'] = 0.0

signals_df['A1_Strategy_Signal'] = np.where((signals_df['RSI14'] > signals_df['RSI_SMA_7']) & (signals_df['volume'] > signals_df['Volume_SMA_7']),
                                1, np.where((signals_df['RSI14'] < signals_df['RSI_SMA_7']) & (signals_df['volume'] < signals_df['Volume_SMA_7']), -1, 0)
)

signals_df['A1_Strategy_Signal']

In [ ]:
# Checking to see how many buy signals compared to sell signals
signals_df['A1_Strategy_Signal'].value_counts()

In [ ]:
# Reviewing the df
signals_df

In [ ]:
# Creating the MACD indicator
MACD_df = TA.MACD(four_hr_ohlcv)
MACD_df

In [ ]:
# Adding the indicators to the signals df
signals_df['MACD'] = MACD_df['MACD']
signals_df['SIGNAL'] = MACD_df['SIGNAL']

In [ ]:
# Reviewing the signals df for the new indicators
signals_df

In [ ]:
# Creating the Squeeze momentum indicator and adding it to the signals df
SQZMI_df = TA.SQZMI(four_hr_ohlcv)
signals_df['SQZMI'] = SQZMI_df
signals_df

In [ ]:
# Creating the ADX indicator and adding it to the signals df
ADX_df = TA.ADX(four_hr_ohlcv)
signals_df['ADX'] = ADX_df

In [ ]:
# Reviewing the df for the new indicator
signals_df

In [ ]:
# Creating the Chandelier indicator, used for stop losses based on the average true range(ATR)
Chandelier_df = TA.CHANDELIER(four_hr_ohlcv)
signals_df['short_stop'] = Chandelier_df['Short.']
signals_df['long_stop'] = Chandelier_df['Long.']

In [ ]:
# Reviewing df
signals_df

In [ ]:
# Plotting the df
signals_df.hvplot(
    height=500,
    width=1000
)

In [ ]:
# Defining a function that calls a set of functions to be created on any df
def indicators(df, sma, rsi, macd_fast, macd_slow, macd_signal, squeeze):

    df['SMA_20'] = TA.SMA(df, sma)
    df['RSI_14'] = TA.RSI(df, rsi)
    MACD = TA.MACD(df, macd_fast,macd_slow,macd_signal,'close', adjust=True)
    df['MACD'] = MACD['MACD']
    df['MACD_Signal'] = MACD['SIGNAL']
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    df['Squeeze'] = TA.SQZMI(df, squeeze)
    return df

In [ ]:
# Creating a function that defines our buy/sell strategy fopr the trading bot
def buy_sell_signals(df, sma_window=20, rsi_window=14, macd_fast=12, macd_slow=26, macd_signal=9, squeeze_window=20):
    
    # Buy signal
    df['signal'] = 0
    df.loc[(df['RSI_14'] < 30) & (df['MACD_Histogram'] > 0) & (df['Squeeze'] == False), 'signal'] = 1

    # Sell signal
    df.loc[(df['RSI_14'] > 70) & (df['MACD_Histogram'] < 0) & (df['Squeeze'] == True), 'signal'] = -1
    
    return df


In [ ]:
# Reviewing the indicator function

indicator_signals_df = indicators(four_hr_ohlcv,20,14,12,26,9,20)
indicator_signals_df

In [ ]:
indicator_signals_df['MACD_buy/sell'] = 0

In [ ]:
indicator_signals_df['Actual_Returns'] = indicator_signals_df['close'].pct_change().dropna()
indicator_signals_df['Signal'] = 0.0
indicator_signals_df.loc[(indicator_signals_df['Actual_Returns'] >= 0), 'Signal'] = 1
indicator_signals_df.loc[(indicator_signals_df['Actual_Returns'] < 0), 'Signal'] = -1

(1 + indicator_signals_df[['Actual_Returns']]).cumprod().hvplot()

In [ ]:
MACD = hv.Curve(indicator_signals_df,'date','MACD').opts(
    title='MACD',
    height=400,
    width=600,
    active_tools=['pan'],
    ylim=(0,1000) 
)
MACD_S = hv.Curve(indicator_signals_df,'date','MACD_Signal').opts(
    title='MACD',
    height=400,
    width=600,
    active_tools=['pan'],
    ylim=(0,1000) 
)
MACD_Histogram = hv.Bars(indicator_signals_df,['date','MACD_Histogram']).opts(
    title='MACD',
    stacked=False,
    multi_level=False,
    height=400,
    width=600,
    active_tools=['pan'],
    ylim=(0,1000) )

curve_rend = hv.render(MACD)
curve_rend_2 = hv.render(MACD_S)
rend = hv.render(MACD_Histogram)
curve_rend.x_range=rend.x_range
curve_rend_2.x_range=rend.x_range
rend.renderers+=curve_rend.renderers
pn.pane.Bokeh(rend).show()



# Machine Learning Section
### Using Ada Boost to train and test data

In [ ]:
indicator_signals_df.info()

In [ ]:
list(indicator_signals_df.dtypes[indicator_signals_df.dtypes=='bool'].index)

In [ ]:
categorical_variables = ['Squeeze']

In [ ]:
# Creating a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(indicator_signals_df[categorical_variables])
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns=enc.get_feature_names(categorical_variables)
)
# Review the DataFrame
encoded_df.head()

In [ ]:
list(indicator_signals_df.dtypes[indicator_signals_df.dtypes=='float64'].index)

In [ ]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables=[]
numerical_variables_df = pd.DataFrame(
    indicator_signals_df,
    columns=(numerical_variables)
)

joined_encoded_df = pd.concat([encoded_df, numerical_variables_df],axis=1)

# Review the DataFrame
joined_encoded_df.head()

In [ ]:
# Creating training and testing data
y = joined_encoded_df['']

X = joined_encoded_df.drop(columns=(''))
# Reviewing testing data
display(y.head())
display(X.head())

In [ ]:
# Splitting the features and target sets into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [ ]:
# Creating a StandardScaler instance
scaler = StandardScaler()
# Fitting the scaler to the features traing dataset
X_scaler - scaler.fit(X_train)
# Fitting the scaled dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaled.transform(X_test)

### Initiating the Deep Neural Network
Using a Deep Neural network to train and test data

In [ ]:
# Defining the number of inputs
number_input_features = len(X_train.iloc[0])

# Reviewing the number of features
display(number_input_features)

# Defining number of outputs
number_output_neurons = 1

# Defining the number of hidden nodes for the first layer
hidden_nodes_layer1 = np.ceil(np.sqrt(number_input_features * number_output_nuerons))

# Reviewing the number of hidden nodes
display(hidden_nodes_layer1)

# Defining the number of hidden nodes for the second layer
hidden_nodes_layer2 = np.ceil(np.sqrt(hidden_nodes_layer1 * number_output_nuerons))

# Reviewing the number of hidden nodes in the second layer
display(hidden_nodes_layer2)

In [ ]:
# Creating the Sequential model instance
nn = Sequential()

In [ ]:
# Adding the first layer
nn.add(
    Dense(
        units=hidden_nodes_layer1,
        activation='relu',
        input_dim=number_input_features
    )
)

# Adding the second layer
nn.add(
    Dense(
        units=hidden_nodes_layer2,
        activation='relu'
    )
)

# Adding the output layer
nn.add(
    Dense(
        units=1,
        activation='sigmoid'
    )
)

In [ ]:
# Reviewing the Sequential model summary
nn.summary()

In [ ]:
# Compiling the Sequential model
nn.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Fitting the model with 100 epochs and the training data
nn.model=nn.fit(X_train, y_train, epochs=100, verbose=2)

### Initiating the Ada Boost Machine learning model

In [ ]:
# Initiate the model instance
ada_boost_model = AdaBoostClassifier()

In [ ]:
# Fit the model using the training data
ada_boost_model = ada_boost_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
ada_boost_pred = ada_boost_model.predict(X_test_scaled)

# Review the model's predicted values
ada_boost_pred

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
ada_testing_report = classification_report(y_test, ada_boost_pred)
# Print the classification report
print(ada_testing_report)

In [ ]:
# Create a predictions DataFrame
ada_predictions_df = pd.DataFrame(index=X_test.index)
# Add the SVM model predictions to the DataFrame
ada_predictions_df['Predicted'] = ada_boost_pred
# Add the actual returns to the DataFrame
ada_predictions_df['Actual Returns'] = indicator_signals_df['Actual Returns']
# Add the strategy returns to the DataFrame
ada_predictions_df['ADA Strategy Returns'] = indicator_signals_df['Actual Returns'] * ada_predictions_df['Predicted']
# Review the DataFrame
display(ada_predictions_df.head())
display(ada_predictions_df.tail())

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + ada_predictions_df[['Actual Returns','ADA Strategy Returns']]).cumprod().plot()

# MAking a stand alone plot for the ADA Model
ADA_plot = (1 + ada_predictions_df[['ADA Strategy Returns']]).cumprod().hvplot()